# Prepare to start <a class="anchor" id="prepare"></a>

## Loading packages

In [1]:
#!pip install git+https://github.com/qubvel/segmentation_models

In [2]:
#!pip install efficientnet-pytorch

In [3]:
import os
import cv2
import glob
import traceback
import cv2 as cv
import numpy as np
import pandas as pd
from path import Path
from tqdm import tqdm
import nibabel as nib
import pydicom as dicom
import tensorflow as tf
from keras import layers
from pydicom import dcmread
from tensorflow import keras
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from pydicom.data import get_testdata_files
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [4]:
original_train_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
original_train_df

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
2014,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1
2015,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0
2017,1.2.826.0.1.3680043.12053,0,0,0,0,0,0,0,0


In [5]:
train_df = pd.read_csv("../input/train-filtered/train_filtered.csv")
train_df.drop('Unnamed: 0', axis=1, inplace=True)
train_df.set_index('index', inplace=True)
train_df.head()

,StudyInstanceUID,Image,Label,Fractured
index,,,,
0,1.2.826.0.1.3680043.780,60.0,1.0,0.0
1,1.2.826.0.1.3680043.780,61.0,1.0,0.0
2,1.2.826.0.1.3680043.780,62.0,1.0,0.0
3,1.2.826.0.1.3680043.780,84.0,2.0,1.0
4,1.2.826.0.1.3680043.780,85.0,2.0,1.0


In [6]:
train_df["Label"].replace(1, "C1", inplace=True)
train_df["Label"].replace(2, "C2", inplace=True)
train_df["Label"].replace(3, "C3", inplace=True)
train_df["Label"].replace(4, "C4", inplace=True)
train_df["Label"].replace(5, "C5", inplace=True)
train_df["Label"].replace(6, "C6", inplace=True)
train_df["Label"].replace(7, "C7", inplace=True)

train_df["Fractured"].replace(0, "not-fractured", inplace=True)
train_df["Fractured"].replace(1, "fractured", inplace=True)

train_df

,StudyInstanceUID,Image,Label,Fractured
index,,,,
0,1.2.826.0.1.3680043.780,60.0,C1,not-fractured
1,1.2.826.0.1.3680043.780,61.0,C1,not-fractured
2,1.2.826.0.1.3680043.780,62.0,C1,not-fractured
3,1.2.826.0.1.3680043.780,84.0,C2,fractured
4,1.2.826.0.1.3680043.780,85.0,C2,fractured
...,...,...,...,...
4915,1.2.826.0.1.3680043.31077,373.0,C7,not-fractured
4916,1.2.826.0.1.3680043.31077,374.0,C7,not-fractured
4917,1.2.826.0.1.3680043.31077,375.0,C7,not-fractured


In [7]:
train_df["Target"]= train_df[["Label", "Fractured"]].apply("-".join, axis=1)
train_df

,StudyInstanceUID,Image,Label,Fractured,Target
index,,,,,
0,1.2.826.0.1.3680043.780,60.0,C1,not-fractured,C1-not-fractured
1,1.2.826.0.1.3680043.780,61.0,C1,not-fractured,C1-not-fractured
2,1.2.826.0.1.3680043.780,62.0,C1,not-fractured,C1-not-fractured
3,1.2.826.0.1.3680043.780,84.0,C2,fractured,C2-fractured
4,1.2.826.0.1.3680043.780,85.0,C2,fractured,C2-fractured
...,...,...,...,...,...
4915,1.2.826.0.1.3680043.31077,373.0,C7,not-fractured,C7-not-fractured
4916,1.2.826.0.1.3680043.31077,374.0,C7,not-fractured,C7-not-fractured
4917,1.2.826.0.1.3680043.31077,375.0,C7,not-fractured,C7-not-fractured


In [8]:
train_df["Target"].replace('C1-fractured', 0, inplace=True)
train_df["Target"].replace('C1-not-fractured', 1, inplace=True)
train_df["Target"].replace('C2-fractured', 2, inplace=True)
train_df["Target"].replace('C2-not-fractured', 3, inplace=True)
train_df["Target"].replace('C3-fractured', 4, inplace=True)
train_df["Target"].replace('C3-not-fractured', 5, inplace=True)
train_df["Target"].replace('C4-fractured', 6, inplace=True)
train_df["Target"].replace('C4-not-fractured', 7, inplace=True)
train_df["Target"].replace('C5-fractured', 8, inplace=True)
train_df["Target"].replace('C5-not-fractured', 9, inplace=True)
train_df["Target"].replace('C6-fractured', 10, inplace=True)
train_df["Target"].replace('C6-not-fractured', 11, inplace=True)
train_df["Target"].replace('C7-fractured', 12, inplace=True)
train_df["Target"].replace('C7-not-fractured', 13, inplace=True)

In [9]:
train_df

,StudyInstanceUID,Image,Label,Fractured,Target
index,,,,,
0,1.2.826.0.1.3680043.780,60.0,C1,not-fractured,1
1,1.2.826.0.1.3680043.780,61.0,C1,not-fractured,1
2,1.2.826.0.1.3680043.780,62.0,C1,not-fractured,1
3,1.2.826.0.1.3680043.780,84.0,C2,fractured,2
4,1.2.826.0.1.3680043.780,85.0,C2,fractured,2
...,...,...,...,...,...
4915,1.2.826.0.1.3680043.31077,373.0,C7,not-fractured,13
4916,1.2.826.0.1.3680043.31077,374.0,C7,not-fractured,13
4917,1.2.826.0.1.3680043.31077,375.0,C7,not-fractured,13


In [10]:
test_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/test.csv")
test_df

,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.10197_C1,1.2.826.0.1.3680043.10197,C1
1,1.2.826.0.1.3680043.10454_C1,1.2.826.0.1.3680043.10454,C1
2,1.2.826.0.1.3680043.10690_C1,1.2.826.0.1.3680043.10690,C1


In [11]:
submission = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv")
submission

,row_id,fractured
0,1.2.826.0.1.3680043.10197_C1,0.5
1,1.2.826.0.1.3680043.10454_C1,0.5
2,1.2.826.0.1.3680043.10690_C1,0.5


In [12]:
def load_dicom(path, size = 64):
    try:
        img=dicom.dcmread(path)
        img.PhotometricInterpretation = 'YBR_FULL'
        data=img.pixel_array
        data=data-np.min(data) #substract the minimum number in the array to all items in the array.
        if np.max(data) != 0:
            data=data/np.max(data) #normalizing.
        data=(data*255).astype(np.uint8)        
        return cv2.cvtColor(data.reshape(512, 512), cv2.COLOR_GRAY2RGB)
    except:        
        return np.zeros((512, 512, 3))

The **YBR FULL 422** photometric interpretation is commonly used with lossy JPEG data.
Each pair of pixels occupies 4 bytes, each pixel having its own luminance (Y) value, but sharing the chroma (R “redness” and B “blueness”) values.
In theory, this is also allowed for uncompressed data, but experience shows that many cheaper viewers do not support it. In uncompressed data, the values are stored as “packed cells”, as follows:

Y Y B R Y Y B R Y Y B R….

In [13]:
def listdirs(folder):
    return [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]

In [14]:
train_dir = '../input/rsna-2022-cervical-spine-fracture-detection/train_images/'
test_dir = '../input/rsna-2022-cervical-spine-fracture-detection/test_images/'
patients = sorted(os.listdir(train_dir))
patients[:5]

['1.2.826.0.1.3680043.10001',
 '1.2.826.0.1.3680043.10005',
 '1.2.826.0.1.3680043.10014',
 '1.2.826.0.1.3680043.10016',
 '1.2.826.0.1.3680043.10032']

For now, we will iterate through our train_filtered file and check if the images are fractured bones or not (target in "Fractured" column).
Next step will be split these images in different folders. Ex. C1-fractured folder + C1-not-fractured folder... etc.

In [15]:
def RSNATrainGenerator(train_df, batch_size, infinite = True, base_path = train_dir):
    while True:
        trainset = []
        trainidt = []
        trainlabel = []
        
        for i in (range(len(train_df))):
            
            idt = train_df.loc[i, 'StudyInstanceUID']
            im = train_df.loc[i, 'Image']
            train_path = train_dir + idt + "/" + str(int(im)+1) + ".dcm" # +1 added because index, started at 0 but actually images names starts from 1.
            
            """Acá chequear que no me haya mandado una cagada con el train_filtered. Tengo imagenes con '0' de nombre y no existen. Quiero hacer referencia
                a la imagen 1?? """
            
            dc = dicom.read_file(train_path)
            if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
                continue
            
            img = load_dicom(train_path)
            img = cv.resize(img, (64 , 64))
            image = img_to_array(img)
            image = image / 255.0
            trainset += [image]

            cur_label = []
            cur_label.append(train_df.loc[i,'Target'])
            trainlabel += [cur_label]
            trainidt += [idt]
            if len(trainidt) == batch_size:                    
                yield np.array(trainset), np.array(trainlabel)
                trainset, trainlabel, trainidt = [], [], []

In [16]:
def RSNATestGenerator(test_df, batch_size, infinite = True, base_path = test_dir):
    while 1:        
        testset=[]
        testidt=[]
        for i in (range(len(test_df))):        
            if type(test_df) is list: idt = test_df[i]
            else: idt = test_df['StudyInstanceUID'].iloc[i]
            path = os.path.join(base_path, idt)
            if os.path.exists(path):
                for im in os.listdir(path):
                    dc = dicom.read_file(os.path.join(path,im))
                    if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
                        continue
                    img=load_dicom(os.path.join(path,im))
                    img=cv.resize(img,(64,64))
                    image=img_to_array(img)
                    image=image/255.0
                    testset+=[image]
                    testidt+=[idt]
                    if len(testset) == batch_size:                        
                        yield np.array(testset)
                        testset = []
        if len(testset) > 0: yield np.array(testset)
        if not infinite: break

In [17]:
def get_model():
    inp = keras.layers.Input((None, None ,1))
    x = Conv2D(3, 3, padding = 'SAME')(inp)
    x = EfficientNetB0(include_top=False)(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    out = keras.layers.Dense(14, 'softmax')(x)
    model = keras.models.Model(inp, out)
    model.summary()
    model.compile(loss="categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001))
    return model

In [18]:
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf

In [19]:
train_df

,StudyInstanceUID,Image,Label,Fractured,Target
index,,,,,
0,1.2.826.0.1.3680043.780,60.0,C1,not-fractured,1
1,1.2.826.0.1.3680043.780,61.0,C1,not-fractured,1
2,1.2.826.0.1.3680043.780,62.0,C1,not-fractured,1
3,1.2.826.0.1.3680043.780,84.0,C2,fractured,2
4,1.2.826.0.1.3680043.780,85.0,C2,fractured,2
...,...,...,...,...,...
4915,1.2.826.0.1.3680043.31077,373.0,C7,not-fractured,13
4916,1.2.826.0.1.3680043.31077,374.0,C7,not-fractured,13
4917,1.2.826.0.1.3680043.31077,375.0,C7,not-fractured,13


In [20]:
for train_idx, val_idx in KFold(5).split(train_df): #train_df contains both train and validation data. validation will be part of the train test    
    K.clear_session()
    x_train = train_df.iloc[train_idx].reset_index()
    x_val = train_df.iloc[val_idx].reset_index()
    model = get_model()
    hist = model.fit(                            
                        RSNATrainGenerator(x_train, min(len(x_train), 64), infinite = False, base_path = train_dir),
                        epochs = 50,
                        verbose = 1,
                        callbacks = [keras.callbacks.EarlyStopping(monitor = 'loss', patience = 2, restore_best_weights = True)],
                        validation_steps = max((len(x_val) // 64), 1),
                        steps_per_epoch = max((len(x_train) // 64), 1),
                        validation_data = RSNATrainGenerator(x_val, min(len(x_val), 64), infinite = False, base_path = train_dir),
                    )
    
    val_pred = model.predict(RSNATestGenerator(x_val, min(len(test_df), 64), infinite = False, base_path = train_dir), steps = max((len(test_df) // 64), 1))    
    try: # the best we can do at the moment..
        preds = model.predict(RSNATestGenerator(test_df, min(len(test_df), 64), infinite = False, base_path = test_dir), steps = max((len(test_df) // 64), 1))
        
        new_preds = []
        for pred_idx in range(len(preds)):
            new_preds.append(preds[pred_idx][prediction_type_mapping[pred_idx]])
        # submission['fractured'] += preds[:, prediction_type_mapping] / 5
        submission['fractured'] += np.array(new_preds) / 5
        
    except: traceback.print_exc()    

2022-08-03 03:03:44.445196: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 03:03:44.562944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 03:03:44.563747: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 03:03:44.566085: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

16719872/16705208 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 3)     30        
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 14)                17934     
Total params: 4,067,535
Trainable params: 4,025,512
Non-trainable params: 42,023
_________________________________________________________________


2022-08-03 03:03:50.802139: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-08-03 03:03:58.989972: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


...

Traceback (most recent call last):
  File "/tmp/ipykernel_23/489923985.py", line 18, in <module>
    preds = model.predict(RSNATestGenerator(test_df, min(len(test_df), 64), infinite = False, base_path = test_dir), steps = max((len(test_df) // 64), 1))
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1730, in predict
    steps_per_execution=self._steps_per_execution)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 1383, in get_data_handler
    return DataHandler(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 1150, in __init__
    model=model)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 794, in __init__
    peek, x = self._peek_and_restore(x)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 851, in _peek_and_restore
    peek = next(x)
StopIteration


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 3)     30        
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 14)                17934     
Total params: 4,067,535
Trainable params: 4,025,512
Non-trainable params: 42,023
_________________________________________________________________
Epoch 1/50
47/47 [==============================] - 35s 613ms/step - loss: 206.3597 - val_loss: 182.4050
Epoch 2

Traceback (most recent call last):
  File "/tmp/ipykernel_23/489923985.py", line 18, in <module>
    preds = model.predict(RSNATestGenerator(test_df, min(len(test_df), 64), infinite = False, base_path = test_dir), steps = max((len(test_df) // 64), 1))
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1730, in predict
    steps_per_execution=self._steps_per_execution)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 1383, in get_data_handler
    return DataHandler(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 1150, in __init__
    model=model)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 794, in __init__
    peek, x = self._peek_and_restore(x)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 851, in _peek_and_restore
    peek = next(x)
StopIteration


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 3)     30        
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 14)                17934     
Total params: 4,067,535
Trainable params: 4,025,512
Non-trainable params: 42,023
_________________________________________________________________
Epoch 1/50
47/47 [==============================] - 35s 616ms/step - loss: 231.6134 - val_loss: 179.5052
Epoch 2

Traceback (most recent call last):
  File "/tmp/ipykernel_23/489923985.py", line 18, in <module>
    preds = model.predict(RSNATestGenerator(test_df, min(len(test_df), 64), infinite = False, base_path = test_dir), steps = max((len(test_df) // 64), 1))
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1730, in predict
    steps_per_execution=self._steps_per_execution)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 1383, in get_data_handler
    return DataHandler(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 1150, in __init__
    model=model)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 794, in __init__
    peek, x = self._peek_and_restore(x)
  File "/opt/conda/lib/python3.7/site-packages/keras/engine/data_adapter.py", line 851, in _peek_and_restore
    peek = next(x)
StopIteration


In [21]:
# outputs.append(means[item[3]])    

In [22]:
submission

,row_id,fractured
0,1.2.826.0.1.3680043.10197_C1,0.5
1,1.2.826.0.1.3680043.10454_C1,0.5
2,1.2.826.0.1.3680043.10690_C1,0.5
